In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import sys
from fxpmath import Fxp

In [2]:
dir_current = os.path.abspath('')
dir_parent  = os.path.dirname(dir_current)
if not dir_parent in sys.path: sys.path.append(dir_parent)

## Parametros de Simulacion

In [3]:
#Parametros de Cuantizacion
bits_activaciones      = 12
precision_activaciones = 6
bits_pesos_conv        = 12
precision_pesos_conv   = 8
bits_pesos_fc          = 6
precision_pesos_fc     = 6

# Parametros de buffer input/output
Malpc = 12    #Maximo de activaciones leidas en un ciclo
Maepc = 12    #Maximo de activaciones escritas en un ciclo
bits_lectura_input    = Malpc*bits_activaciones
bits_escritura_output = Maepc*bits_activaciones

#Parametros de la matriz de dsp slices.
filas      = Malpc
dsp_slices = 720
columnas   = dsp_slices/filas

#Parametros de Buffers de pesos
Mpclpc = 24   # Maximo de pesos convolucionales leidos en un ciclo
Mpflpc = 48   # Maximo de pesos full conectados leidos en un ciclo
bits_lectura_pesos_conv  = Mpclpc*bits_pesos_conv
bits_lectura_pesos_fc    = Mpflpc*bits_pesos_fc

#Tamaño del buffer
IOBuffer_size = 3211264*bits_activaciones

## Carga del modelo y sus pesos

In [4]:
from models import VGG16_body
from models import Weight_Quantization

# Creacion de la red
Nclasses = 9

Qinput_layer  = tf.keras.Input((224,224,3))
Qoutput_layer = VGG16_body(Qinput_layer, Quantization = True, word_size = bits_activaciones, frac_size = precision_activaciones, N_labels = Nclasses)
QVGG16 = tf.keras.Model(inputs=Qinput_layer, outputs=Qoutput_layer)

# Carga de los pesos
cwd = os.getcwd()
Wgt_dir = os.path.join(cwd,'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir,'WeedWeights')
Wgt_dir = os.path.join(Wgt_dir,'Weights')
QVGG16.load_weights(Wgt_dir)

# Cuantizacion de los pesos
Weight_Quantization(model = QVGG16, 
                    Frac_Bits = precision_pesos_conv, Int_Bits = (bits_pesos_conv-precision_pesos_conv-1),
                    Dense_Frac_Bits = precision_pesos_fc, Dense_Int_Bits = (bits_pesos_fc-precision_pesos_fc-1))

## Carga del dataset

In [13]:
from models import Quantization_layer
from functions import VGG_resize_v2

x_test,  _  = tfds.as_numpy(tfds.load('deep_weeds', split='train[85%:86%]', batch_size=-1, as_supervised=True))
del _
# Normalizacion
x_test = x_test[0:30]
x_test = x_test/255.
# Cuantizacion
x_test = Quantization_layer(x_test)

test_dataset = tf.data.Dataset.from_tensor_slices(x_test)
test_dataset = test_dataset.map(VGG_resize_v2)

In [2]:
# Funcion para correr la simulacion remotamente

In [16]:
from functions import get_all_outputs
def get_Batch_outputs(model, iterator, List_of_layers):
    List  = []
    image = next(iterator,None)
    while image != None:
        Outputs = get_all_outputs(model, tf.expand_dims(image,axis=0))
        Outputs = [Outputs[i] for i in List_of_layers]
        List.append(Outputs)
        image = next(iterator,None)
    return List

In [17]:
LayerList = [1,3,5,7,9,10,12,14,16,18,19,21,23,25,27,29,31,32,34,36,38,40,42,44,45,47,49,51,53,55,57,58,61,63,65,67,69,71]
iterator  = iter(test_dataset)
ActList   = get_Batch_outputs(QVGG16, iterator, LayerList)

## Simulacion

In [6]:
# Bufferes
IOBuffer_1 = np.zeros(IOBuffer_size,dtype=int)
IOBuffer_2 = np.zeros(IOBuffer_size,dtype=int)

# Diccionario de estadisticas de los buffers
stats_IOBuffer_1 = {"ultimo_valor"    : np.zeros(IOBuffer_size,dtype=int),
                    "ciclos_1"        : np.zeros(IOBuffer_size,dtype=int),
                    "cambios_logicos" :np.zeros(IOBuffer_size,dtype=int)}
stats_IOBuffer_2 = {"ultimo_valor"    : np.zeros(IOBuffer_size,dtype=int),
                    "ciclos_1"        : np.zeros(IOBuffer_size,dtype=int),
                    "cambios_logicos" :np.zeros(IOBuffer_size,dtype=int)}

import simulation
import time
from functions import save_obj, load_obj, Load_Image

# Variables de simulacion
simulation.Malpc            = Malpc
simulation.Maepc            = Maepc
simulation.Mpclpc           = Mpclpc
simulation.Mpflpc           = Mpflpc
simulation.filas            = filas
simulation.columnas         = columnas
simulation.Word_bits        = bits_activaciones
simulation.Frac_bits        = precision_activaciones
simulation.QVGG16           = QVGG16
simulation.IOBuffer_1       = IOBuffer_1
simulation.stats_IOBuffer_1 = stats_IOBuffer_1
simulation.IOBuffer_2       = IOBuffer_2
simulation.stats_IOBuffer_2 = stats_IOBuffer_2


iterator   = iter(test_dataset)
ciclos     = 0
index      = 0
batch_size = 1
t = time.time()
while index < batch_size:
    image = next(iterator)
    Load_Image(IOBuffer_1,image, Word_bits = bits_activaciones, Frac_bits = precision_activaciones)
    ciclos += simulation.VGG16_Simulation_b(image)
    save_obj(stats_IOBuffer_1,'stats_B1')
    save_obj(stats_IOBuffer_2,'stats_B2')
    index = index + 1
    print('imagenes procesadas:',index,' tiempo:',time.time()-t,'ciclos: ',ciclos)
print('Tiempo de simulacion:',time.time()-t)
save_obj(ciclos,'numero_de_ciclos')

inicia simulacion
Capa Convolucional
tiempo transcurrido: 17713.968396663666 ciclos: 309016
Capa Convolucional
tiempo transcurrido: 39515.74285316467 ciclos: 3606735
Capa MaxPooling
tiempo transcurrido: 50622.528431892395 ciclos: 3678415
Capa Convolucional
tiempo transcurrido: 66777.32387661934 ciclos: 5210512
Capa Convolucional
tiempo transcurrido: 86304.19273376465 ciclos: 8274769
Capa MaxPooling
tiempo transcurrido: 90359.7020213604 ciclos: 8310609
Capa Convolucional
tiempo transcurrido: 98319.44616127014 ciclos: 9758239
Capa Convolucional
tiempo transcurrido: 105659.11774849892 ciclos: 12653549
Capa Convolucional
tiempo transcurrido: 113487.66587257385 ciclos: 15548859
Capa MaxPooling
tiempo transcurrido: 115621.49722123146 ciclos: 15570363
Capa Convolucional
tiempo transcurrido: 120349.53251171112 ciclos: 17367416
Capa Convolucional
tiempo transcurrido: 125467.5360686779 ciclos: 20961589
Capa Convolucional
tiempo transcurrido: 131505.03171944618 ciclos: 24555762
Capa MaxPooling
ti